In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyvallocation.views import BlackLittermanProcessor
from pyvallocation.portfolioapi import AssetsDistribution, PortfolioWrapper
from pyvallocation.utils.projection import  project_mean_covariance, log2simple

# load daily close data for some ETFs
df = pd.read_csv('../examples/ETF_prices.csv',index_col=0,parse_dates=True)
print('Loaded ', df.columns)

# resample to weekly frequency
weekly_prices = df.resample('W').ffill()

# compute compounded returns
weekly_returns = np.log(weekly_prices).diff().dropna()

# store the returns shape
T, N = weekly_returns.shape

# inputs/parameters
ANNUALIZATION_FACTOR = 52
HALF_LIFE = ANNUALIZATION_FACTOR * 10 # 10 years
TARGET_RETURN = 0.07/ANNUALIZATION_FACTOR

INVESTMENT_HORIZON = 26 # weeks ahead
TARGET_RETURN_HORIZON = TARGET_RETURN*INVESTMENT_HORIZON

Loaded  Index(['DBC', 'GLD', 'SPY', 'TLT'], dtype='object')


In [2]:
def print_portfolio(name: str, weights: pd.Series, ret: float, risk: float, risk_measure: str):
    """Prints formatted portfolio details."""
    print(f"\n--- {name} ---")
    print(f"  > Expected Return: {ret:.4f}")
    print(f"  > {risk_measure}: {risk:.4f}")
    print("  > Weights:")
    print(weights.round(3).to_string())
    print("-" * 25)

## Estimate sample moments

In [3]:
from pyvallocation import moments, probabilities
p = probabilities.generate_uniform_probabilities(T)
mu, cov = moments.estimate_sample_moments(weekly_returns,p)

# Apply Jorion's shirnkage
mu_ = moments.shrink_mean_jorion(mu,cov,T)

# Apply Ledoit&Wolf's shrinkage
cov_ = moments.shrink_covariance_ledoit_wolf(weekly_returns,cov,target='identity')

# Project the first two moments to horizon
mu_hor, cov_hor = project_mean_covariance(mu_,cov_,INVESTMENT_HORIZON)

# Convert in simple returns
mu_hor_simple, cov_hor_simple = log2simple(mu_hor,cov_hor)

/Users/giuliocantadori/dev/Py-vAllocation/pyvallocation/moments.py:73: RuntimeWarning: divide by zero encountered in matmul
  S = (X.T * p_arr) @ X
/Users/giuliocantadori/dev/Py-vAllocation/pyvallocation/moments.py:73: RuntimeWarning: overflow encountered in matmul
  S = (X.T * p_arr) @ X
/Users/giuliocantadori/dev/Py-vAllocation/pyvallocation/moments.py:73: RuntimeWarning: invalid value encountered in matmul
  S = (X.T * p_arr) @ X


## MVO Portfolio

In [4]:

simple_hor_dist = AssetsDistribution(mu=mu_hor_simple,cov=cov_hor_simple)
    
# Step 2: Initialize the wrapper
simple_wrapper = PortfolioWrapper(simple_hor_dist)

# Step 3: Set constraints (long-only, fully invested)
simple_wrapper.set_constraints({'long_only': True, 'total_weight': 1.0})

# Step 4: Compute the efficient frontier
simple_frontier = simple_wrapper.mean_variance_frontier(num_portfolios=20)

print(f"\nComputed a Mean-Variance frontier with {simple_frontier.weights.shape[1]} portfolios.")
print(f"Risk measure used: '{simple_frontier.risk_measure}'")

w_min_ret, r_min_ret, est_min_ret = simple_frontier.portfolio_at_return_target(TARGET_RETURN_HORIZON)
print_portfolio(f"MVO Portfolio with Minimum Return {TARGET_RETURN_HORIZON}", w_min_ret, r_min_ret, est_min_ret, simple_frontier.risk_measure)

[pyvallocation.portfolioapi - INFO] PortfolioWrapper initialized for 4 assets.
[pyvallocation.portfolioapi - INFO] Setting constraints with parameters: {'long_only': True, 'total_weight': 1.0}
[pyvallocation.portfolioapi - INFO] Successfully computed Mean-Variance frontier with 20 portfolios.



Computed a Mean-Variance frontier with 20 portfolios.
Risk measure used: 'Volatility'

--- MVO Portfolio with Minimum Return 0.035 ---
  > Expected Return: 0.0359
  > Volatility: 0.0664
  > Weights:
DBC    0.036
GLD    0.223
SPY    0.330
TLT    0.411
-------------------------


## Black and Litterman

In [5]:
mu_bl, cov_bl = BlackLittermanProcessor(
    prior_mean=mu_,
    prior_cov=cov_,
    mean_views={'SPY':-0.05/ANNUALIZATION_FACTOR},
    view_confidences=1,
    # tau=1,
    idzorek_use_tau=False,
    omega='idzorek',
    verbose=True,
).get_posterior()

print(mu*ANNUALIZATION_FACTOR)
print(mu_bl*ANNUALIZATION_FACTOR)
mu_bl_hor,cov_bl_hor = log2simple(*project_mean_covariance(mu_bl,cov_bl,INVESTMENT_HORIZON))


bl_hor_dist = AssetsDistribution(mu=mu_bl_hor,cov=cov_bl_hor)
    
# Step 2: Initialize the wrapper
bl_wrapper = PortfolioWrapper(bl_hor_dist)

# Step 3: Set constraints (long-only, fully invested)
bl_wrapper.set_constraints({'long_only': True, 'total_weight': 1.0})

# Step 4: Compute the efficient frontier
bl_frontier = bl_wrapper.mean_variance_frontier(num_portfolios=50)

print(f"\nComputed a Mean-Variance frontier with {bl_frontier.weights.shape[1]} portfolios.")
print(f"Risk measure used: '{bl_frontier.risk_measure}'")

[pyvallocation.portfolioapi - INFO] PortfolioWrapper initialized for 4 assets.
[pyvallocation.portfolioapi - INFO] Setting constraints with parameters: {'long_only': True, 'total_weight': 1.0}


[BL] π source: prior_mean.
[BL] Built P (1, 4), Q (1, 1).
[BL] Ω from Idzorek confidences (base = Σ).
[BL] Posterior mean and covariance computed.
DBC    0.006164
GLD    0.089540
SPY    0.097284
TLT    0.027575
Name: mu, dtype: float64
DBC   -0.033581
GLD    0.063792
SPY   -0.049997
TLT    0.060012
dtype: float64


[pyvallocation.portfolioapi - INFO] Successfully computed Mean-Variance frontier with 50 portfolios.



Computed a Mean-Variance frontier with 50 portfolios.
Risk measure used: 'Volatility'


In [6]:
w_bl_min_risk, r_bl_min_risk, est_bl_min_risk = bl_frontier.get_min_risk_portfolio()
print_portfolio("BL Portfolio with Minimum Risk", w_bl_min_risk, r_bl_min_risk, est_bl_min_risk, bl_frontier.risk_measure)


--- BL Portfolio with Minimum Risk ---
  > Expected Return: 0.0153
  > Volatility: 0.0636
  > Weights:
DBC    0.174
GLD    0.094
SPY    0.259
TLT    0.472
-------------------------


In [7]:
w_bl_min_ret, r_bl_min_ret, est_bl_min_ret = bl_frontier.portfolio_at_return_target(TARGET_RETURN_HORIZON)
print_portfolio(f"BL Portfolio with Minimum Return {TARGET_RETURN_HORIZON}", w_bl_min_ret, r_bl_min_ret, est_bl_min_ret, bl_frontier.risk_measure)


--- BL Portfolio with Minimum Return 0.035 ---
  > Expected Return: 0.0352
  > Volatility: 0.0834
  > Weights:
DBC    0.000
GLD    0.390
SPY    0.046
TLT    0.563
-------------------------


## NIW posteriors

In [8]:
from pyvallocation.bayesian import NIWPosterior

niw_updater = NIWPosterior(mu_bl_hor,cov_bl_hor,T/2,T/2)
posterior_params= niw_updater.update(mu_hor_simple,cov_hor_simple,T)
est_expected_return = posterior_params.mu1         # This is μ₁ from Meucci
est_uncertainty_cov = posterior_params.sigma1     # This is Σ₁ from Meucci

# For gamma-variant specific parameters:
p_mu_conf = .9  # Example confidence level for mean uncertainty
gamma_mu_val = niw_updater.cred_radius_mu(p_mu=p_mu_conf) # This is γμ
print(gamma_mu_val)

0.0718484744295931


In [9]:
print('Sample mean projected at horizon:\n',mu_hor)
print('\nBL mean project at horizon\n',mu_bl_hor)
print('\nNIW mean projected at horizon\n', est_expected_return)

Sample mean projected at horizon:
 DBC    0.011410
GLD    0.036095
SPY    0.038387
TLT    0.017749
Name: mu, dtype: float64

BL mean project at horizon
 DBC   -0.007170
GLD    0.040366
SPY   -0.016498
TLT    0.035927
dtype: float64

NIW mean projected at horizon
 DBC    0.011504
GLD    0.043031
SPY    0.026407
TLT    0.027352
dtype: float64


## Robust Optimization

In [10]:
# Step 1: Create distribution with posterior parameters
robust_dist = AssetsDistribution(mu=est_expected_return, cov=est_uncertainty_cov)

# Step 2: Initialize wrapper
robust_wrapper = PortfolioWrapper(robust_dist)

# Step 3: Set constraints
robust_wrapper.set_constraints({'long_only': True, 'total_weight': 1.0})

# --- 3a: Lambda-Frontier ---
print("\n--- Solving for the Robust λ-Frontier ---")

# Step 4a: Compute the λ-frontier
# This frontier plots Nominal Return vs. Estimation Risk (‖Σ'¹/²w‖₂)
lambda_frontier = robust_wrapper.robust_lambda_frontier(num_portfolios=30, max_lambda=1.0)

print(f"\nComputed a Robust λ-frontier with {lambda_frontier.weights.shape[1]} portfolios.")
print(f"Risk measure used: '{lambda_frontier.risk_measure}'")

# Step 5a: Analyze the λ-frontier
# We can find the portfolio that minimizes estimation uncertainty
w_min_est_risk, r_min_est_risk, est_risk_min = lambda_frontier.get_min_risk_portfolio()
print_portfolio("Portfolio with Minimum Estimation Risk", w_min_est_risk, r_min_est_risk, est_risk_min, lambda_frontier.risk_measure)

w_lambda_min_ret, r_lambda_min_ret, est_lambda_min_ret = lambda_frontier.portfolio_at_return_target(TARGET_RETURN_HORIZON)
print_portfolio(f"BL Portfolio with Minimum Return {TARGET_RETURN_HORIZON}",w_lambda_min_ret, r_lambda_min_ret, est_lambda_min_ret, bl_frontier.risk_measure)

[pyvallocation.portfolioapi - INFO] PortfolioWrapper initialized for 4 assets.
[pyvallocation.portfolioapi - INFO] Setting constraints with parameters: {'long_only': True, 'total_weight': 1.0}
[pyvallocation.portfolioapi - INFO] Computing robust λ-frontier. Assuming `dist.mu` is posterior mean (μ₁) and `dist.cov` is posterior scale matrix (Σ₁), used as uncertainty covariance (Σ').



--- Solving for the Robust λ-Frontier ---


[pyvallocation.portfolioapi - INFO] Successfully computed Robust λ-frontier with 30 portfolios.



Computed a Robust λ-frontier with 30 portfolios.
Risk measure used: 'Estimation Risk (‖Σ'¹/²w‖₂)'

--- Portfolio with Minimum Estimation Risk ---
  > Expected Return: 0.0318
  > Estimation Risk (‖Σ'¹/²w‖₂): 0.0642
  > Weights:
DBC    0.004
GLD    0.307
SPY    0.319
TLT    0.370
-------------------------

--- BL Portfolio with Minimum Return 0.035 ---
  > Expected Return: 0.0354
  > Volatility: 0.0737
  > Weights:
DBC    0.000
GLD    0.530
SPY    0.268
TLT    0.202
-------------------------


In [11]:
# --- 3b: Gamma-Portfolio ---
print("\n--- Solving for a single Robust γ-Portfolio ---")

gamma_sigma_sq_val = 0.1**2 # Squared upper bound on estimation risk

# Step 4b: Solve for the single optimal portfolio
w_gamma, r_gamma, est_risk_gamma = robust_wrapper.solve_robust_gamma_portfolio(
    gamma_mu=gamma_mu_val, 
    gamma_sigma_sq=gamma_sigma_sq_val
)

print_portfolio(
    f"Robust γ-Portfolio (γ_μ={gamma_mu_val}, γ_σ²={gamma_sigma_sq_val})",
    w_gamma,
    r_gamma,
    est_risk_gamma,
    "Estimation Risk (‖Σ'¹/²w‖₂)"
)

[pyvallocation.portfolioapi - INFO] Solving robust γ-portfolio. Assuming `dist.mu` is posterior mean (μ₁) and `dist.cov` is posterior scale matrix (Σ₁), used as uncertainty covariance (Σ').
[pyvallocation.portfolioapi - INFO] Successfully solved robust γ-portfolio. Nominal Return: 0.0402, Estimation Risk: 0.1000



--- Solving for a single Robust γ-Portfolio ---

--- Robust γ-Portfolio (γ_μ=0.0718484744295931, γ_σ²=0.010000000000000002) ---
  > Expected Return: 0.0402
  > Estimation Risk (‖Σ'¹/²w‖₂): 0.1000
  > Weights:
DBC    0.000
GLD    0.831
SPY    0.169
TLT    0.000
-------------------------
